# Building a Recommendation System in PySpark - Lab

## Introduction

In this lab, we will implement a movie recommendation system using ALS in Spark programming environment. Spark's machine learning library `ml` comes packaged with a very efficient implementation of the ALS algorithm that we looked at in the previous lesson. The lab will require you to put into practice your Spark programming skills for creating and manipulating PySpark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and PySpark using the `MovieLens` dataset that we used in a previous lab.

Note: You are advised to refer to [PySpark documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 


## Objectives

In this lab you will: 

- Use Spark to train and cross-validate an ALS model 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 

## Building a Recommendation System

We have seen how recommendation systems have played an integral part in the success of Amazon (books, items), Pandora/Spotify (music), Google (news, search), YouTube (videos) etc.  For Amazon, these systems bring more than 30% of their total revenue. For Netflix, 75% of movies that people watch are based on some sort of recommendation.

> The goal of recommendation systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.


For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of a recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad 
2.    User B performs a search query for Game of Thrones 
3.    The system suggests Breaking Bad to user B from data collected about user A 


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the `MovieLens` dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Squares implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

### Importing the Data

* Initialize a `SparkSession` object
* Import the dataset found at `'./data/ratings.csv'` into a PySpark DataFrame

In [100]:
# import necessary libraries
from pyspark.sql import SparkSession
# instantiate SparkSession object
# spark = SparkSession.builder.master("local").getOrCreate()
spark = SparkSession.builder.master('local').getOrCreate()

In [4]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv("./data/ratings.csv", header="true", inferSchema="true")

Check the data types of each of the columns to ensure that they are a type that makes sense given the column.

In [6]:
movie_ratings.dtypes

[('userId', 'int'),
 ('movieId', 'int'),
 ('rating', 'double'),
 ('timestamp', 'int')]

We aren't going to need the timestamp, so we can go ahead and remove that column.

In [7]:
movie_ratings = movie_ratings.drop('timestamp')
movie_ratings.dtypes

[('userId', 'int'), ('movieId', 'int'), ('rating', 'double')]

### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Import `ALS` from `pyspark.ml.recommendation` module 
* Use the `.randomSplit()` method on the pyspark DataFrame to separate the dataset into training and test sets
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the `userCol`, `itemCol`, and `ratingCol` to the appropriate columns given this dataset. Then fit the data to the training set and assign it to a variable model  

In [8]:
from pyspark.ml.recommendation import ALS 

# split into training and testing sets
train, test = movie_ratings.randomSplit(weights=[0.8, 0.2], seed=1)

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(userCol= 'userId', itemCol='movieId', ratingCol = 'rating', coldStartStrategy='drop')
# fit the ALS model to the training set
model = als.fit(train)

24/01/14 11:45:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/01/14 11:45:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/01/14 11:45:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/01/14 11:45:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* Import `RegressionEvalutor` from `pyspark.ml.evaluation` 
* Generate predictions with your model for the test set by using the `.transform()` method on your ALS model 
* Evaluate your model and print out the RMSE from your test set 

In [11]:
# importing appropriate library
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Evaluate the model by computing the RMSE on the test data
preds = model.transform(test)
rmse = evaluator.evaluate(preds)

print("Root Mean Squared Error (RMSE) on test data =", rmse)

Root Mean Squared Error (RMSE) on test data = 0.878404938984475


### Cross-validation to Find the Optimal Model

Let's now find the optimal values for the parameters of the ALS model. Use the built-in `CrossValidator` in PySpark with a suitable param grid and determine the optimal model. Try with the parameters:

* regularization = [0.01, 0.001, 0.1]
* rank = [4, 10, 50]


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# initialize the ALS model
als = ALS(userCol= 'userId', itemCol='movieId', ratingCol = 'rating', coldStartStrategy='drop')
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# create the parameter grid              
params = (ParamGridBuilder()
          .addGrid(als.regParam, [0.01, 0.001, 0.1])
          .addGrid(als.rank, [4, 10, 50])
          .build())


# instantiating crossvalidator estimator
cv = CrossValidator(estimator=als,
                    estimatorParamMaps=params,
                    evaluator=evaluator)

cvModel = cv.fit(movie_ratings)

bestModel = cvModel.bestModel

bestScore = min(cvModel.avgMetrics)

bestParams = bestModel.extractParamMap()

print("Best Model Score (RMSE):", bestScore)
print("Best Model Parameters:")
for param, value in bestParams.items():
    print(f"{param.name}: {value}")

print("Best Model Parameters:")
for param in [als.regParam, als.rank]:
    print(f"{param.name}: {bestParams[param]}")

# We see the best model has a rank of 50, so we will use that in our future models with this dataset

In [57]:
print("Best Model Score (RMSE):", bestScore)
print("Best Model Parameters:")
for param, value in bestParams.items():
    print(f"{param.name}: {value}")

Best Model Score (RMSE): 0.8957251106557234
Best Model Parameters:
predictionCol: prediction
blockSize: 4096
coldStartStrategy: drop
itemCol: movieId
userCol: userId


In [58]:
print("Best Model Parameters:")
for param in [als.regParam, als.rank]:
    print(f"{param.name}: {bestParams[param]}")

Best Model Parameters:


KeyError: Param(parent='ALS_6444dcfcbb6c', name='regParam', doc='regularization parameter (>= 0).')

In [59]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialize the ALS model
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop')
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Create the parameter grid
params = (ParamGridBuilder()
          .addGrid(als.regParam, [0.01, 0.001, 0.1])
          .addGrid(als.rank, [4, 10, 50])
          .build())

# Instantiate CrossValidator estimator
cv = CrossValidator(estimator=als,
                    estimatorParamMaps=params,
                    evaluator=evaluator)

cvModel = cv.fit(movie_ratings)

bestModel = cvModel.bestModel
bestScore = min(cvModel.avgMetrics)
bestParams = bestModel.extractParamMap()

print("Best Model Score (RMSE):", bestScore)
print("Best Model Parameters:")
for param in [als.regParam, als.rank]:
    print(f"{param.name}: {bestParams[param]}")



Best Model Score (RMSE): 0.8957251106557234
Best Model Parameters:


KeyError: Param(parent='ALS_6fd4c7e24d1e', name='regParam', doc='regularization parameter (>= 0).')

### Incorporating the names of the movies

When we make recommendations, it would be ideal if we could have the actual name of the movie used rather than just an ID. There is another file called `'./data/movies.csv'` that contains all of the names of the movies matched up to the `movie_id` that we have in the ratings dataset.

* Import the data into a Spark DataFrame 
* Look at the first 5 rows

In [65]:
movie_titles = spark.read.csv('./data/movies.csv', header="true", inferSchema="true")

movie_titles.head(5)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId=2, title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId=4, title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId=5, title='Father of the Bride Part II (1995)', genres='Comedy')]

We will eventually be matching up the movie ids with the movie titles. In the cell below, create a function `name_retriever()` that takes in a `movie_id` and returns a string that represents the movie title. 

> Hint: It's possible to do this operation in one line with the `df.where()` or the `df.filter()` methods 

In [82]:
def name_retriever(movie_id, movie_title_df):
    return movie_title_df.filter(movie_title_df.movieId == movie_id).select('title').collect()[0]['title']

In [83]:
print(name_retriever(1023, movie_titles))

Winnie the Pooh and the Blustery Day (1968)


## Getting Recommendations

Now it's time to actually get some recommendations! The ALS model has built-in methods called `.recommendForUserSubset()` and `.recommendForAllUsers()`. We'll start off with using a subset of users. 

In [89]:
model = bestModel
users = movie_ratings.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs = model.recommendForUserSubset(users, 10)
recs = userSubsetRecs.take(1)
recs

[Row(userId=148, recommendations=[Row(movieId=98491, rating=4.566890239715576), Row(movieId=160718, rating=4.394077301025391), Row(movieId=33649, rating=4.376354694366455), Row(movieId=40629, rating=4.143664360046387), Row(movieId=7121, rating=4.130632400512695), Row(movieId=142488, rating=4.12627649307251), Row(movieId=30816, rating=4.103631019592285), Row(movieId=183897, rating=4.081643581390381), Row(movieId=122918, rating=4.038437366485596), Row(movieId=98243, rating=4.0353875160217285)])]

We can now see we have a list of rows with recommended items. Now try and get the name of the top recommended movie by way of the function you just created, using number one item for this user.

In [96]:
# use indexing to obtain the movie id of top predicted rated item
first_recommendation = recs[0]['recommendations'][0][0]

# use the name retriever function to get the values
name_retriever(first_recommendation,movie_titles)

'Paperman (2012)'

Of course, you can also make recommendations for everyone, although this will take longer. In the next line, we are creating an RDD with the top 5 recommendations for every user and then selecting one user to find out his predictions:

In [97]:
recommendations = model.recommendForAllUsers(5)
recommendations.where(recommendations.userId == 3).collect()

[Row(userId=3, recommendations=[Row(movieId=70946, rating=4.9636688232421875), Row(movieId=5746, rating=4.917801380157471), Row(movieId=6835, rating=4.917801380157471), Row(movieId=5919, rating=4.868616580963135), Row(movieId=5181, rating=4.858239650726318)])]

### Getting Predictions for a New User

Now, it's time to put together all that you've learned in this section to create a function that will take in a new user and some movies they've rated and then return $n$ number of highest recommended movies. This function will have multiple different steps to it:

* Adding the new ratings into the DataFrame (hint: look into using the `.union()` method) 
* Fitting the ALS model  
* Make recommendations for the user of choice 
* Print out the names of the top $n$ recommendations in a reader-friendly manner 

The function should take in the parameters: 

* `user_id` : int 
* `new_ratings` : list of tuples in the format (user_id, item_id, rating)
* `rating_df` : spark DF containing ratings
* `movie_title_df` : spark DF containing movie titles
* `num_recs` : int

Rate new movies

```python
[Row(movieId=3253, title='Wayne's World (1992)', genres='Comedy'),
 Row(movieId=2459, title='Texas Chainsaw Massacre, The (1974)', genres='Horror'),
 Row(movieId=2513, title='Pet Sematary (1989)', genres='Horror'),
 Row(movieId=6502, title='28 Days Later (2002)', genres='Action|Horror|Sci-Fi'),
 Row(movieId=1091, title='Weekend at Bernie's (1989)', genres='Comedy'),
Row(movieId=441, title='Dazed and Confused (1993)', genres='Comedy'),
Row(movieId=370, title='Naked Gun 33 1/3: The Final Insult (1994)', genres='Action|Comedy')]

```

In [134]:
from pyspark.sql import Row
user_id = 666
new_ratings = Ron_ratings
rating_df = movie_ratings
movie_title_df = movie_titles
num_recs = 10

new_rating_rows = [Row(user_id=i[0], movieId=i[1], rating=i[2]) for i in new_ratings]

new_rating_df = spark.createDataFrame(new_rating_rows)
    
# combine the new ratings df with the rating_df
updated_rating_df = rating_df.union(new_rating_df)

# # create an ALS model and fit it
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop', rank=50)
model = als.fit(updated_rating_df)

# # make recommendations for all users using the recommendForAllUsers method
# recommendations = model.recommendForAllUsers(num_recs)


# # get recommendations specifically for the new user that has been added to the DataFrame
# user_rec_ids = recommendations.where(recommendations.userId == user_id).collect()[0]['recommendations']
# num = 0
# print(f'Top {num_recs} Movie Recommendations')
# for i in user_rec_ids:
#     num += 1
#     id = i[0]
#     title = name_retriever(id,movie_title_df)
#     print(f'{num}. {title}')

In [140]:
def new_user_recs(user_id, new_ratings, rating_df, movie_title_df, num_recs):
    # turn the new_recommendations list into a spark DataFrame
    new_rating_rows = [Row(user_id=i[0], movieId=i[1], rating=i[2]) for i in new_ratings]

    new_rating_df = spark.createDataFrame(new_rating_rows)
        
    # combine the new ratings df with the rating_df
    updated_rating_df = rating_df.union(new_rating_df)
    
    # create an ALS model and fit it
    als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop', rank=50)
    model = als.fit(updated_rating_df)
    
    # make recommendations for all users using the recommendForAllUsers method
    recommendations = model.recommendForAllUsers(num_recs)


    # get recommendations specifically for the new user that has been added to the DataFrame
    user_rec_ids = recommendations.where(recommendations.userId == user_id).collect()[0]['recommendations']
    num = 0
    print(f'Top {num_recs} Movie Recommendations')
    for i in user_rec_ids:
        num += 1
        id = i[0]
        title = name_retriever(id,movie_title_df)
        print(f'{num}. {title}')
        

In [137]:
# make recommendations for all users using the recommendForAllUsers method
recommendations = model.recommendForAllUsers(num_recs)


# get recommendations specifically for the new user that has been added to the DataFrame
user_rec_ids = recommendations.where(recommendations.userId == user_id).collect()[0]['recommendations']
num = 0
print(f'Top {num_recs} Movie Recommendations')
for i in user_rec_ids:
    num += 1
    id = i[0]
    title = name_retriever(id,movie_title_df)
    print(f'{num}. {title}')

Top 10 Movie Recommendations
1. Eat Drink Man Woman (Yin shi nan nu) (1994)
2. Texas Chainsaw Massacre, The (1974)
3. Seventh Seal, The (Sjunde inseglet, Det) (1957)
4. Shine (1996)
5. Victory (a.k.a. Escape to Victory) (1981)
6. Patton (1970)
7. Weekend at Bernie's (1989)
8. Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)
9. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
10. Saving Face (2004)


In [142]:
# try out your function with the movies listed above
Ron_ratings = [(666, 3253, 3),
               (666, 2459, 5),
               (666, 2513, 3),
               (666, 6502, 5),
               (666, 1091, 5),
               (666, 441, 3),
               (666, 370, 2)]

new_user_recs(666, Ron_ratings, movie_ratings, movie_titles, 10)

Top 10 Movie Recommendations
1. Eat Drink Man Woman (Yin shi nan nu) (1994)
2. Texas Chainsaw Massacre, The (1974)
3. Seventh Seal, The (Sjunde inseglet, Det) (1957)
4. Shine (1996)
5. Victory (a.k.a. Escape to Victory) (1981)
6. Patton (1970)
7. Weekend at Bernie's (1989)
8. Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)
9. Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
10. Saving Face (2004)


So here we have it! Our recommendation system is generating recommendations for the top 10 movies. 




## Level up (Optional)


* Create a user interface to allow users to easily choose items and get recommendations 

* Use IMDB links to scrape user reviews from IMDB and using basic NLP techniques, create extra embeddings for ALS model  

* Create a hybrid recommender system using features like genre 

## Summary

In this lab, you built a model using Spark, performed some parameter selection, and updated the model every time new user preferences came in. You looked at how Spark's ALS implementation can be used to build a scalable and efficient recommendation system. You also saw that such systems can become computationally expensive and using them with an online system could be a problem with traditional computational platforms. Spark's distributed computing architecture provides a great solution to deploy such recommendation systems for real-world applications (think Amazon, Spotify).